In [ ]:
import cv2
import numpy as np
import math

# Global variables
points = []
dragging_point = -1  # Keeps track of which point is being dragged
image = None
clone = None

def calculate_angle(pt1, pt2, pt3):
    """Calculate the angle between two lines formed by three points."""
    v1 = [pt1[0] - pt2[0], pt1[1] - pt2[1]]
    v2 = [pt3[0] - pt2[0], pt3[1] - pt2[1]]
    
    # Calculate the angle in radians and convert it to degrees
    angle_rad = math.atan2(v2[1], v2[0]) - math.atan2(v1[1], v1[0])
    angle_deg = math.degrees(angle_rad)
    
    # Ensure the angle is positive
    if angle_deg < 0:
        angle_deg += 360
    return angle_deg

def draw_points_and_lines(img, points):
    """Draw points and lines based on the selected points."""
    # Draw circles on each selected point
    for point in points:
        cv2.circle(img, point, 5, (0, 0, 255), -1)

    # Draw lines if enough points are selected
    if len(points) >= 2:
        pt1, pt2 = points[0], points[1]
        cv2.line(img, pt1, pt2, (255, 0, 0), 2)  # First line
    if len(points) == 3:
        pt2, pt3 = points[1], points[2]
        cv2.line(img, pt2, pt3, (255, 0, 0), 2)  # Second line

        # Calculate the angle and display it
        angle = calculate_angle(points[0], points[1], points[2])
        cv2.putText(img, f"Angle: {angle:.2f} degrees", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    return img

def find_nearest_point(x, y, points, threshold=10):
    """Find the nearest point to (x, y) from the list of points."""
    for i, point in enumerate(points):
        if abs(point[0] - x) < threshold and abs(point[1] - y) < threshold:
            return i
    return -1

def mouse_callback(event, x, y, flags, param):
    """Handle mouse events for selecting, dragging, and moving points."""
    global points, dragging_point, image, clone
    
    if event == cv2.EVENT_LBUTTONDOWN:
        # Check if user is clicking near an existing point to drag
        dragging_point = find_nearest_point(x, y, points)
        if dragging_point == -1 and len(points) < 3:
            # If no point is nearby and less than 3 points, add a new point
            points.append((x, y))
            print(f"Point selected: ({x}, {y})")
            # If three points are selected, calculate and print the angle
            if len(points) == 3:
                angle = calculate_angle(points[0], points[1], points[2])
                print(f"Angle between the 3 points: {angle:.2f} degrees")

        # Update and display the image immediately after point selection
        image = clone.copy()
        image = draw_points_and_lines(image, points)
        cv2.imshow("Angle Measurement", image)
    
    elif event == cv2.EVENT_MOUSEMOVE and dragging_point != -1:
        # If dragging, move the selected point
        points[dragging_point] = (x, y)
        image = clone.copy()
        image = draw_points_and_lines(image, points)
        
        # Calculate and print the angle when moving points
        if len(points) == 3:
            angle = calculate_angle(points[0], points[1], points[2])
            print(f"Updated angle: {angle:.2f} degrees")

        cv2.imshow("Angle Measurement", image)

    elif event == cv2.EVENT_LBUTTONUP:
        # Stop dragging
        dragging_point = -1

def interactive_angle_measure(image_path):
    """Open a window for interactive angle measurement using OpenCV."""
    global image, clone, points

    # Load and display the image
    image = cv2.imread(image_path)
    clone = image.copy()

    cv2.imshow("Angle Measurement", image)
    cv2.setMouseCallback("Angle Measurement", mouse_callback)

    while True:
        # Wait for the user to press the 'r' key to reset or 'q' to quit
        key = cv2.waitKey(1) & 0xFF

        # Reset the image if 'r' is pressed
        if key == ord('r'):
            points = []
            image = clone.copy()
            cv2.imshow("Angle Measurement", image)

        # Break the loop if 'q' is pressed
        elif key == ord('q'):
            break

    cv2.destroyAllWindows()



# Example usage
image_path = '/Users/mohamedbasuony/Desktop/letter_e.jpg'
interactive_angle_measure(image_path)


Point selected: (458, 461)
Point selected: (460, 414)
Point selected: (304, 267)
Angle between the 3 points: 130.86 degrees
Updated angle: 137.67 degrees
Updated angle: 147.43 degrees
Updated angle: 157.26 degrees
Updated angle: 176.08 degrees
Updated angle: 176.08 degrees
Updated angle: 176.08 degrees
Updated angle: 176.08 degrees
Updated angle: 176.09 degrees
Updated angle: 176.13 degrees
Updated angle: 176.15 degrees
Updated angle: 176.20 degrees
Updated angle: 176.23 degrees
Updated angle: 176.29 degrees
Updated angle: 176.33 degrees
Updated angle: 176.33 degrees
Updated angle: 176.34 degrees
Updated angle: 176.34 degrees
Updated angle: 176.34 degrees
Updated angle: 176.36 degrees
Updated angle: 176.36 degrees
Updated angle: 176.36 degrees
Updated angle: 176.36 degrees


In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import random
import svgwrite

# Function to generate random colors
def random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Class for the drawing application
class DrawingApp:
    def __init__(self, root, image_path):
        self.root = root
        self.root.title("Drawing App")
        
        # Load the image
        self.image = Image.open(image_path)
        self.tk_image = ImageTk.PhotoImage(self.image)
        
        # Create a canvas and add the image to it
        self.canvas = tk.Canvas(root, width=self.image.width, height=self.image.height, bg="white")
        self.canvas.pack()

        # Add image to the canvas
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)
        
        # Initialize drawing-related variables
        self.last_x, self.last_y = None, None
        self.stroke_color = random_color()
        self.strokes = []  # To store all drawn strokes

        # Bind mouse events
        self.canvas.bind("<Button-1>", self.on_click)
        self.canvas.bind("<B1-Motion>", self.on_drag)

        # Bind the 'S' key for saving
        self.root.bind("<s>", self.save_as_svg)

    def on_click(self, event):
        self.last_x, self.last_y = event.x, event.y
        self.stroke_color = random_color()  # New random color for each stroke
        self.current_stroke = [(event.x, event.y)]  # Start a new stroke

    def on_drag(self, event):
        x, y = event.x, event.y
        # Draw the line segment
        self.canvas.create_line(self.last_x, self.last_y, x, y, fill=self.stroke_color, width=2)
        # Store the segment in the current stroke
        self.current_stroke.append((x, y))
        self.last_x, self.last_y = x, y

    def on_release(self, event):
        # Finalize the stroke
        self.strokes.append((self.current_stroke, self.stroke_color))
        self.current_stroke = []

    def save_as_svg(self, event=None):
        # Ask user for save location
        file_path = filedialog.asksaveasfilename(defaultextension=".svg", filetypes=[("SVG files", "*.svg")])
        if not file_path:
            return  # User canceled save

        # Create an SVG file
        dwg = svgwrite.Drawing(file_path, profile='tiny', size=(self.image.width, self.image.height))
        
        # Add all strokes to the SVG
        for stroke, color in self.strokes:
            dwg.add(dwg.polyline(stroke, stroke_linejoin='round', stroke=color, fill='none', stroke_width=2))
        
        # Save the SVG
        dwg.save()
        print(f"SVG saved to {file_path}")

# Main function to run the application
def main(image_path):
    root = tk.Tk()
    app = DrawingApp(root, image_path)
    root.mainloop()

# Run the application with the image path
if __name__ == "__main__":
    image_path = "/Users/mohamedbasuony/Desktop/letter_e.jpg"  # Replace with your image file path
    main(image_path)

In [ ]:
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter
import cv2
import numpy as np

# Function to preprocess the image for better OCR results
def preprocess_image(image_path):
    # Open the image using Pillow
    image = Image.open(image_path)

    # Convert image to grayscale
    gray_image = image.convert('L')

    # Enhance contrast
    enhancer = ImageEnhance.Contrast(gray_image)
    enhanced_image = enhancer.enhance(2.0)  # Increase contrast

    # Apply thresholding to remove noise and make text clearer
    threshold_image = enhanced_image.point(lambda p: p > 128 and 255)

    return threshold_image

# Function to perform OCR and find the line containing the text
def extract_text_line(image_path, search_text):
    # Preprocess the image for better OCR results
    preprocessed_image = preprocess_image(image_path)

    # Perform OCR with pytesseract, using the 'words' output option to get bounding boxes
    custom_config = r'--oem 3 --psm 6'  # Default OEM, single uniform block of text
    data = pytesseract.image_to_data(preprocessed_image, config=custom_config, output_type=pytesseract.Output.DICT)

    # Initialize variables to track the location of the line with the search text
    found = False
    line_bbox = None

    # Combine words on the same line to form full sentences and compare with the search text
    for i, word in enumerate(data['text']):
        if search_text.lower() in word.lower():
            # Get the bounding box of the line
            left = data['left'][i]
            top = data['top'][i]
            width = data['width'][i]
            height = data['height'][i]
            line_bbox = (left, top, left + width, top + height)
            found = True
            break

    if not found:
        print(f"Text '{search_text}' not found in the image.")
        return None

    # Crop the image using the bounding box of the text line
    cropped_image = preprocessed_image.crop(line_bbox)

    # Return the cropped image (line screenshot)
    return cropped_image

# Main function to handle inputs and save the result
def main(image_path, search_text, output_path):
    result_image = extract_text_line(image_path, search_text)

    if result_image:
        # Save the cropped line screenshot
        result_image.save(output_path)
        print(f"Screenshot of the line containing '{search_text}' saved as {output_path}")
    else:
        print(f"Text '{search_text}' not found, no image saved.")

    
# Example usage
if __name__ == "__main__":
    manuscript_image = "/Users/mohamedbasuony/Desktop/maxresdefault.jpg"  # Path to the image of the manuscript
    text_to_find = "One"  # Text to search for
    output_screenshot = "/Users/mohamedbasuony/Desktop/output_line_screenshot.png"  # Path to save the cropped screenshot

    main(manuscript_image, text_to_find, output_screenshot)